In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import sys

In [34]:
import pandas as pd
import sys
import plotly.express as px

data = pd.read_csv('Reports/FullReport.csv')

data['time'] = pd.to_datetime(data['time'], unit='s')

# Ask the user to choose the date
user_choice = input("Enter 'today' for today's date, or enter a date in YYYY-MM-DD format: ")

if user_choice.lower() == 'today':
    selected_date = pd.to_datetime('today').date()
else:
    try:
        selected_date = pd.to_datetime(user_choice, format='%Y-%m-%d').date()
    except ValueError:
        print("Invalid date format. Please enter 'today' or a valid date in the format YYYY-MM-DD.")
        sys.exit(1)

# Filter data for the selected date
selected_data = data[data['time'].dt.date == selected_date]

# Now update the 'time' column in the selected_data DataFrame
selected_data = selected_data.assign(
    time=selected_data['time'].dt.strftime('%H:%M')
)

# Check if data is available for the selected date
if selected_data.empty:
    print(f"No data available for the selected date ({selected_date}). Please choose another date.")
    sys.exit(1)

# Choose the number of intervals to display on the X-axis
max_intervals = 20

# Calculate the step size to skip data points
step_size = max(1, len(selected_data) // max_intervals)

# Extract the subset of data for X-axis labels
x_labels_data = selected_data.iloc[::step_size]

# Plotting Battery Percentage with Plotly
fig = px.line(selected_data, x='time', y='batteryPercent', labels={'batteryPercent': 'Battery Percentage'})
fig.update_traces(line=dict(color='#DCCBC3'))
fig.update_layout(
    title=f'Battery Percentage on {selected_date.strftime("%Y-%m-%d")}',
    xaxis_title='Time',
    yaxis_title='Battery Percentage',
    xaxis=dict(tickvals=x_labels_data['time'].values, ticktext=x_labels_data['time'].values),
    yaxis=dict(range=[0, 100]),
    plot_bgcolor='#FEF9FF',
    margin_pad=20
)
fig.show()

# Plotting Memory Usage of Processes with Plotly
data_sorted_by_mem = selected_data.sort_values(by='hungryProcMem', ascending=False)
fig = px.bar(data_sorted_by_mem, x='hungriestProc', y='hungryProcMem', labels={'hungryProcMem': 'Memory Usage (MB)'})
fig.update_traces(marker=dict(color='#B97375'))
fig.update_layout(
    title=f'Memory Usage of Processes on {selected_date.strftime("%Y-%m-%d")}',
    xaxis_title='Processes',
    yaxis_title='Memory Usage (MB)',
    xaxis=dict(tickangle=45),
    barmode='relative',
    barnorm='percent',
    plot_bgcolor='#FEF9FF',
)
fig.show()

# Plotting Network Activity with Plotly
fig = px.line(selected_data, x='time', y='bytesSent', labels={'value': 'Bytes'}, line_shape='linear')
fig.add_scatter(x=selected_data['time'], y=selected_data['bytesReceived'], mode='lines', name='bytesReceived', line=dict(color='#C4929A'))
fig.add_scatter(x=selected_data['time'], y=selected_data['bytesSent'], mode='lines', name='bytesSent', line=dict(color='#E2DCDE'))
fig.update_layout(
    title=f'Network Activity on {selected_date.strftime("%Y-%m-%d")}',
    xaxis_title='Time',
    yaxis_title='Bytes',
    xaxis=dict(tickvals=x_labels_data['time'].values, ticktext=x_labels_data['time'].values),
    plot_bgcolor='#FEF9FF',
)
fig.show()

# Plotting Running Processes with Plotly
fig = px.line(selected_data, x='time', y='runningProc', labels={'runningProc': 'Number of Running Processes'})
fig.update_traces(line=dict(color='#BAB0ED'))
fig.update_layout(
    title=f'Running Processes on {selected_date.strftime("%Y-%m-%d")}',
    xaxis_title='Time',
    yaxis_title='Number of Running Processes',
    xaxis=dict(tickvals=x_labels_data['time'].values, ticktext=x_labels_data['time'].values),
    plot_bgcolor='#FEF9FF',
)
fig.show()

# Calculate memory usage percentage for each process
total_memory = selected_data['memUsageMB'].sum()
selected_data['memory_percentage'] = (selected_data['memUsageMB'] / total_memory) * 100

# Group by process name and calculate total memory usage percentage for each process
process_memory_percentage = selected_data.groupby('hungriestProc')['memory_percentage'].sum().reset_index()

custom_colors = ['#B97375', '#CEB1BE', '#F1E4E8', '#E2DCDE', '#2D2D34']


# Plotting the pie chart for memory usage of processes
fig = px.pie(process_memory_percentage,
             values='memory_percentage',
             names='hungriestProc',
             title=f'Memory Usage of Processes on {selected_date.strftime("%Y-%m-%d")}',
            color_discrete_sequence=custom_colors
)
fig.update_traces(textposition='inside', textinfo='label')
fig.show()
